# Curso: Big Data - Tarea 1
## Alonso Nuñez
## Fecha de entrega: 25 noviembre 2019

Importación de dependencias

In [95]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

import findspark
findspark.init('c:\spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

from pyspark.sql.functions import lit

import pandas as pd
from pandas import read_csv

Creación del esquema (estructura de columnas a usar para crear el DataFrame)

In [10]:
schema=StructType().add("id_tienda","string").add("tipo_tienda","string").add("provincia", "string").add("supervisor",'string').add("mes_del_anio", "integer").add("semana_del_anio", "integer").add("es_quincena", "string").add("es_feriado", "string").add("dia_de_la_semana", "string").add("horas_operacion", "integer").add("pos_en_uso", "integer").add("clientes_totales", "integer").add("clientes_vip", "float").add("volumen", "string")


dataset cargado


Crear el dataframe a partir de un csv leido por url, y usando el esquema anterior

In [11]:
clientes = 'https://raw.githubusercontent.com/anunezcr/datasets/master/clientes_diarios_clasif_todasSucursales.csv'
dataset =  pd.read_csv(clientes, sep = ',')
print("dataset cargado")

df=spark.createDataFrame(dataset, schema=schema)

Visualizar el esquema del DataFrame:

In [12]:
df.printSchema()

root
 |-- id_tienda: string (nullable = true)
 |-- tipo_tienda: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- supervisor: string (nullable = true)
 |-- mes_del_anio: integer (nullable = true)
 |-- semana_del_anio: integer (nullable = true)
 |-- es_quincena: string (nullable = true)
 |-- es_feriado: string (nullable = true)
 |-- dia_de_la_semana: string (nullable = true)
 |-- horas_operacion: integer (nullable = true)
 |-- pos_en_uso: integer (nullable = true)
 |-- clientes_totales: integer (nullable = true)
 |-- clientes_vip: float (nullable = true)
 |-- volumen: string (nullable = true)



Visualizar el DataFrame:

In [13]:
df.show(2)

+---------+-----------+---------+-----------+------------+---------------+-----------+----------+----------------+---------------+----------+----------------+------------+-------+
|id_tienda|tipo_tienda|provincia| supervisor|mes_del_anio|semana_del_anio|es_quincena|es_feriado|dia_de_la_semana|horas_operacion|pos_en_uso|clientes_totales|clientes_vip|volumen|
+---------+-----------+---------+-----------+------------+---------------+-----------+----------+----------------+---------------+----------+----------------+------------+-------+
|       12|         FS| SAN JOSE|2015622_080|           1|              1| VERDADERO |Anio Nuevo|           Lunes|             11|         4|             210|        23.0|  medio|
|       12|         FS| SAN JOSE|2015622_080|           1|              1| VERDADERO |       NO |          Martes|             11|         4|             306|        17.0|   alto|
+---------+-----------+---------+-----------+------------+---------------+-----------+----------+---

Comienza análisis de datos:

In [14]:
df.count()

22633

In [15]:
len(df.columns)

14

Mostrar solo unas columnas y filas:

In [18]:
df.select(['dia_de_la_semana','horas_operacion','clientes_totales']).show()

+----------------+---------------+----------------+
|dia_de_la_semana|horas_operacion|clientes_totales|
+----------------+---------------+----------------+
|           Lunes|             11|             210|
|          Martes|             11|             306|
|       Miercoles|             11|             203|
|          Jueves|             11|             192|
|         Viernes|             12|             229|
|          Sabado|             15|             225|
|         Domingo|             16|             224|
|           Lunes|             12|             179|
|          Martes|             11|             268|
|       Miercoles|             10|             215|
|          Jueves|             11|             235|
|         Viernes|             11|             286|
|          Sabado|             14|             250|
|         Domingo|             13|             203|
|           Lunes|             11|             208|
|          Martes|             11|             239|
|       Mier

# Operaciones vistas en clase:

## 1. Mostrar solo ciertos datos a partir de un criterio, y filtrado con comando & (and)

In [35]:
print('Registros con 500 o más clientes diarios por local: ',df.filter(df['clientes_totales'] > 500).count())

Registros con 500 o más clientes diarios por local:  738


In [34]:
print('El ', df.filter(df['clientes_totales'] > 500).count() / df.count()*100, '% de los registros corresponde a 500 clientes o más por local en un solo día')

El  3.2607254893297397 % de los registros corresponde a 500 clientes o más en un solo día


500 clientes diarios, separado por día de la semana:

In [49]:
print('500 o más clientes por día de la semana\n')
print('Lunes: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Lunes')).count())
print('Martes: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Martes')).count())
print('Miercoles: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Miercoles')).count())
print('Jueves: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Jueves')).count())
print('Viernes: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Viernes')).count())
print('Sabado: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Sabado')).count())
print('Domingo: ', df.where((df['clientes_totales'] > 500) & (df['dia_de_la_semana'] == 'Domingo')).count())

500 o más clientes por día de la semana

Lunes:  44
Martes:  54
Miercoles:  38
Jueves:  57
Viernes:  215
Sabado:  262
Domingo:  68


### Conclusión:
Considerando un día de 500 clientes diarios como un día de alto volumen de ventas, tenemos:

Los Viernes y Sabados son los días en que las tiendas necesitan mayor cantidad de personal para atender la demanda de clientes
Los Domingos aunque están en el 3er lugar están lejos de lo que significa un viernes o sábado
Por lo tanto, es incorrecto pensar que los 3 días del fin de semana se requiere la misma cantidad de personal
La empresa podría incrementar productividad y ahorrar costos, si toma esta consideración y deja de pensar en los 3 días de fin de semana como iguales

El domingo (68) está más cercano a un jueves o martes (54-57), que a un sábado a domingo (215-262)

## 2. Contador por categorías

In [50]:
df.groupBy('horas_operacion').count().show()

+---------------+-----+
|horas_operacion|count|
+---------------+-----+
|             12| 3347|
|             22|    7|
|              1|    9|
|             13| 1890|
|             16| 1211|
|              6|  114|
|              3|   17|
|             20|   10|
|              5|   38|
|             19|   47|
|             15| 1155|
|              9| 1807|
|             17| 1271|
|              4|   18|
|              8|  631|
|             23|    5|
|              7|  302|
|             10| 4266|
|             21|   15|
|             11| 4366|
+---------------+-----+
only showing top 20 rows



### Análisis por horas de operación:

Son pocos los registros de tiendas con operación igual o mayor a 19 horas (menos de 75 registros de los 22633)
Estoy podría hacer pensar a la gerencia si conviene o no operar tiendas 24 horas.
Convendría un análisis de costo/beneficio

In [51]:
df.groupBy('pos_en_uso').count().show()

+----------+-----+
|pos_en_uso|count|
+----------+-----+
|         1|  244|
|         6| 2866|
|         3| 3689|
|         5| 4406|
|         4| 8137|
|         8|    7|
|         7|  953|
|         2| 2274|
|         0|   57|
+----------+-----+



### Análisis por cantidad de cajas registradoras:

Se encuentran registros desde 0 hasta 8 cajas operando por día
Los extremos de 0 y 8 se pueden considerar excepciones por el bajo porcentaje
La moda es 4 cajas usadas por día

Esta información podría ser útil en una eventual optimización de recursos, por ejemplo tiendas que tienen 8 cajas registradoras, podría reducirse la cantidad, reduciendo así los inventarios de equipos o reutilizandolos en aperturas futuras reduciendo el presupuesto de inversión

In [54]:
df.groupBy('volumen').count().show()

+-------+-----+
|volumen|count|
+-------+-----+
|   bajo| 1069|
|  medio|14342|
|   alto| 7222|
+-------+-----+



Con los criterios ya definidos en rangos para determinar volumen bajo, medio, alto, se puede observar que la mayoría de registros son volumen medio, seguido por los altos

## 3. Promedio de clientes_totales agrupados por dia_de_la_semana o por mes_del_anio

In [52]:
df.groupBy('dia_de_la_semana').agg(F.mean('clientes_totales')).show()

+----------------+---------------------+
|dia_de_la_semana|avg(clientes_totales)|
+----------------+---------------------+
|          Martes|   246.28420726031646|
|       Miercoles|   233.02789832610043|
|          Jueves|   238.79292145296492|
|           Lunes|   218.76520681265208|
|         Domingo|    265.7223427331887|
|          Sabado|    317.2076875387477|
|         Viernes|   298.79795158286777|
+----------------+---------------------+



In [53]:
df.groupBy('mes_del_anio').agg(F.mean('clientes_totales')).show()

+------------+---------------------+
|mes_del_anio|avg(clientes_totales)|
+------------+---------------------+
|          12|   289.66580976863753|
|           1|   258.76054138469544|
|           6|    256.3344086021505|
|           3|    255.1371220020855|
|           5|   243.70083246618105|
|           9|    262.3462365591398|
|           4|   250.93870967741935|
|           8|   264.06607700312173|
|           7|    263.6649323621228|
|          10|   246.24557752341312|
|          11|    268.3945799457995|
|           2|   256.34274193548384|
+------------+---------------------+



### Promedio de clientes por día de la semana o por mes del año

Podría ser de interés para conocer estacionalidad, y tomar acciones en diferentes aspectos como suministros de materia prima, necesidad de personal, etc.

## 4. Mostrar los datos un orden ascendente o decendente: 

In [60]:
df.sort("es_feriado").show(5)

+---------+-----------+---------+-----------+------------+---------------+-----------+--------------------+----------------+---------------+----------+----------------+------------+-------+
|id_tienda|tipo_tienda|provincia| supervisor|mes_del_anio|semana_del_anio|es_quincena|          es_feriado|dia_de_la_semana|horas_operacion|pos_en_uso|clientes_totales|clientes_vip|volumen|
+---------+-----------+---------+-----------+------------+---------------+-----------+--------------------+----------------+---------------+----------+----------------+------------+-------+
|       13|         IS| SAN JOSE|1972863_080|           7|             30|     FALSO | Anexion del Part...|       Miercoles|             12|         4|             159|        13.0|  medio|
|       12|         FS| SAN JOSE|2015622_080|           7|             30|     FALSO | Anexion del Part...|       Miercoles|             12|         4|             202|        24.0|  medio|
|       14|         FS| SAN JOSE|2015622_080|     

In [61]:
df.sort("horas_operacion", ascending=True).show(5)

+---------+-----------+---------+-----------+------------+---------------+-----------+------------+----------------+---------------+----------+----------------+------------+-------+
|id_tienda|tipo_tienda|provincia| supervisor|mes_del_anio|semana_del_anio|es_quincena|  es_feriado|dia_de_la_semana|horas_operacion|pos_en_uso|clientes_totales|clientes_vip|volumen|
+---------+-----------+---------+-----------+------------+---------------+-----------+------------+----------------+---------------+----------+----------------+------------+-------+
|      100|         FS|    LIMON|1670344_080|          12|             53| VERDADERO |         NO |           Lunes|              0|         0|             212|        47.0|  medio|
|      104|         FS| SAN JOSE|2027736_080|           1|              1|     FALSO |         NO |         Viernes|              0|         4|             173|        12.0|  medio|
|      104|         FS| SAN JOSE|2027736_080|           1|              1| VERDADERO | Ani

In [63]:
df.sort("pos_en_uso", ascending=False).show(5)

+---------+-----------+---------+-----------+------------+---------------+-----------+----------+----------------+---------------+----------+----------------+------------+-------+
|id_tienda|tipo_tienda|provincia| supervisor|mes_del_anio|semana_del_anio|es_quincena|es_feriado|dia_de_la_semana|horas_operacion|pos_en_uso|clientes_totales|clientes_vip|volumen|
+---------+-----------+---------+-----------+------------+---------------+-----------+----------+----------------+---------------+----------+----------------+------------+-------+
|       18|         FS| SAN JOSE|1670344_080|           6|             24|     FALSO |       NO |           Lunes|              9|         8|             353|        15.0|   alto|
|       14|         FS| SAN JOSE|2015622_080|          11|             44| VERDADERO |       NO |         Viernes|             12|         8|             440|        25.0|   alto|
|       18|         FS| SAN JOSE|1670344_080|           5|             18| VERDADERO |       NO |   

## 5. Collect (visto en clase)

In [62]:
df.groupby("clientes_totales").agg(F.collect_set("provincia")).show(10)

+----------------+----------------------+
|clientes_totales|collect_set(provincia)|
+----------------+----------------------+
|             148|  [ALAJUELA, CARTAG...|
|             463|            [SAN JOSE]|
|             471|  [ALAJUELA, GUANAC...|
|             496|            [SAN JOSE]|
|             243|  [ALAJUELA, CARTAG...|
|             392|  [ALAJUELA, CARTAG...|
|             540|  [ALAJUELA, HEREDI...|
|             623|            [SAN JOSE]|
|             737|            [SAN JOSE]|
|             516|  [ALAJUELA, SAN JOSE]|
+----------------+----------------------+
only showing top 10 rows



# Operaciones nuevas (vistas en clase) :

## 1. Filtrado con comando OR
Registros en los días viernes O(or) sábados, que anteriormente vimos eran los más importantes en cantidad de clientes_totales, Y(and) que tengan 500 clientes o más

In [83]:

print('Viernes y Sabado: ', df.where(((df['dia_de_la_semana'] == 'Viernes') | (df['dia_de_la_semana'] == 'Sabado')) & (df['clientes_totales'] > 500)).count())

Viernes y Sabado:  477


## 2. Filtrado con comando not equal (!=)
Mismo filtro anterior pero con registros distintos a viernes y sábado

In [84]:
print('de Domingo a Jueves: ', df.where(((df['dia_de_la_semana'] != 'Viernes') & (df['dia_de_la_semana'] != 'Sabado')) & (df['clientes_totales'] > 500)).count())

de Domingo a Jueves:  261


Los viernes y sábado representan el 64% del total de registros que tienen más de 500 clientes.
Es decir, en solo 2 días de la semana (el 28% del tiempo) se concentra el 64% del volumen de ventas

## 3. Calcular un nuevo valor a partir de columnas existentes (alias)

Por ejemplo: para cada registro, calcular la proporción de clientes_vip sobre clientes_totales

In [134]:

df1 = df.select(['clientes_vip','clientes_totales',(df.clientes_vip/df.clientes_totales).alias("proporcion")])
df1.show()


+------------+----------------+--------------------+
|clientes_vip|clientes_totales|          proporcion|
+------------+----------------+--------------------+
|        23.0|             210| 0.10952380952380952|
|        17.0|             306| 0.05555555555555555|
|        12.0|             203|0.059113300492610835|
|        13.0|             192| 0.06770833333333333|
|        16.0|             229| 0.06986899563318777|
|        22.0|             225| 0.09777777777777778|
|        26.0|             224| 0.11607142857142858|
|        14.0|             179|  0.0782122905027933|
|        15.0|             268|0.055970149253731345|
|        15.0|             215| 0.06976744186046512|
|        19.0|             235| 0.08085106382978724|
|        22.0|             286| 0.07692307692307693|
|        35.0|             250|                0.14|
|        27.0|             203|  0.1330049261083744|
|        18.0|             208| 0.08653846153846154|
|        19.0|             239|  0.07949790794

In [138]:
print('Registros con proporcion mayor a 0.10: ',df1.filter(df1['proporcion'] > 0.10).count())


Registros con proporcion mayor a 0.10:  4716


### Análisis de la proporción de clientes vip sobre clientes totales
Solo en 4716 de los registros los clientes vip son el 10% (o más) del total de clientes
Eso es un 21% del total de registros

Es decir, el 21% de los días, la participación de clientes vip es 10% o más
Vale la pena un análisis mayor sobre esta participación, para valorar si conviene aumentarla, conociendo que los VIP compran más